In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)

from tqdm import tqdm

import requests
import time

In [1]:
# scrapping reddit posts via api.pushshift. scrapping both submissions and comments. they have different pushshift urls.
submission_url = 'https://api.pushshift.io/reddit/search/submission'
comment_url = 'https://api.pushshift.io/reddit/search/comment'

In [3]:
def reddit_scrapper(url, subreddit, epoch, n_iters, pickle_file):
    url = url
    params = {'subreddit': subreddit,
              'size': 100, #100 is pushshifts max
              'before': 1646063999, #1646063999 is 28 Feb 2022. this can be updated or commented out
              'after': epoch # 
             } 
    request_data = []

    for i in tqdm(range(n_iters)):
        if len(request_data) > 0:
            params['after'] = request_data[-1]['created_utc']

        request_data_add = requests.get(url, params).json()['data']
        request_data += request_data_add
        time.sleep(3)

    return pd.DataFrame(request_data).to_pickle(pickle_file)

In [13]:
%%time
# reddit_scrapper function requires (url) either submission or comment pushshift url, (epoch) epoch time to start scrapping from, (n_iters) the iterations of how many 100 posts to scrap, (pickle_file) the pickle file name to save data
reddit_scrapper(submission_url, 'sephora', 1645142400, 50, 'datasets/xxx.pkl')


100%|███████████████████████████████████████████| 50/50 [03:59<00:00,  4.79s/it]

CPU times: user 1.47 s, sys: 186 ms, total: 1.66 s
Wall time: 3min 59s


In [ ]:
# after scrapping, read the pickle file and convert pickle in csv. check for duplicates before saving over. 
df = pd.read_pickle('./datasets/xxx.pkl')
print(df.shape)
df.tail()

pd.DataFrame(df).to_csv('datasets/xxx.csv', index=False)
df = pd.read_csv('./datasets/', low_memory=False)

duplicateRowsDF = df[df.duplicated()]
duplicateRowsDF

df.drop_duplicates(subset=None, keep= 'first', inplace=True)
df.shape

pd.DataFrame(df).to_csv('datasets/xxx.csv', index=False)